# Prediction with LSTM model (v.4.4)
<p style='text-align: right;'>with selectd4.csv</p>

* history
  * 2025/05/23 PM06:02 : 3번째 모델의 3번째 데이터를 사용
  * 2025/05/27

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [14]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

restart_kernel()

restart kernel... 2025-05-28 10:21:30.111322


In [1]:
# load dependacies
import os
import time
import resource
from datetime import datetime
from lib.globalvar import *
from lib.data_loader import DataLoader
from lib.small_predict import predict_small, print_predict_small
from lib.activation import ActivationOutput, RecurrentActivation
from lib.small_predict import build_and_predict, gen_multi_model, save_model


print(f"imported library. ({datetime.now()})")

imported library. (2025-05-28 10:51:11.548010)


In [2]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./csv/selectd4.csv'
    
mode='predict' # help = "back-test or predict")

mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=0.95 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[1, 5, 18, 20, 30, 35],
        [7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 3

write_to_db = True


finished to set environemnt. (2025-05-28 10:51:11.554238)


In [3]:
import tensorflow as tf
print(f'{tf.version}')

<module 'tensorflow._api.v2.version' from '/home/swhors/jupyter-workspace/finance/venv/lib/python3.11/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [4]:
dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0,
                  verbose=False
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300,
                  verbose=False
                  )
    }
#print(f'test_X = {dataset_dicts[1].test_X}')
print(f'completed to load data. {datetime.now()}')

completed to load data. 2025-05-28 10:51:13.678608


In [5]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
datasets = [dataset_dicts[1], # 0
            dataset_dicts[2], # 1
           ]

lstm_units=[[(45, ActivationOutput.selu.name),   # 0
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 1
             (45, ActivationOutput.selu.name)
             ],
            [(45, ActivationOutput.selu.name),   # 2
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.selu.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.sigmoid.name),
             (45, ActivationOutput.elu.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 3
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name)
             ],
            [(45, ActivationOutput.tanh.name),   # 4
             (45, ActivationOutput.tanh.name),
             (45, ActivationOutput.tanh.name),
             ],
            [(45, ActivationOutput.tanh.name)    # 5
             ],
            ]
dense_units = [[(45, ActivationOutput.selu.name),   # 0
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.selu.name),], # 1
               [(45, ActivationOutput.selu.name),   # 2
                (45, ActivationOutput.elu.name),
                (45, ActivationOutput.elu.name),],
               [(45, ActivationOutput.tanh.name),   # 3
               ],
               []                                   # 4
              ]
sel_date = datetime(2025, 5, 24, 20, 35)
sel_date_ts = (sel_date.timestamp() / 10000000)
steps = [45, 45,]
metrics = [["accuracy"], []]
dropout = [0, 0]
learning_rate = [0.01, 0.05]
last_lstm_return_sequences = [False, True]
loss = ["binary_crossentropy", "mse"]
output_dense_activation = [ActivationOutput.elu.name, #0
                           ActivationOutput.selu.name, #1
                           ActivationOutput.sigmoid.name, #2
                           ActivationOutput.tanh.name, #3
                           ActivationOutput.softmax.name, #4
                           ActivationOutput.relu.name, #5
                           ActivationOutput.linear.name #6
                          ]
epochs = [10, #0
          20, #1
          25, #2
          50, #3
          100] #4

rand_seed=[sel_date_ts, datetime.now().timestamp(), 0]


lstm_args = {6: {"model":{"learning_rate": learning_rate[0],
                          "dropout": dropout[0],
                          "steps": steps[0], # last output elements count
                          "metrics": metrics[0],
                          "last_lstm_return_sequences": last_lstm_return_sequences[1],
                          "lstm_units": lstm_units[5],
                          "dense_units": dense_units[4],
                          "output_dense_activation": output_dense_activation[3],
                          "loss": loss[1],
                          "return_state": False,
                          "lstm_model": "Bidirectional",
                          "rand_seed": rand_seed[0] # rand_seed[0]
                          },
                 "train": {"epochs": epochs[1],
                           "is_shuffle": False,
                           "batch": 3,
                           "steps_per_epoch": 0 #100
                           },
                 "dataset": datasets[0]
                 }
             }


print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-28 10:51:15.140051


In [ ]:
#
# max_model_limit 정보만큼 반복적으로 매트릭을 생성합니다.
#

model_version_limit = 20
for version in range(1, model_version_limit+1):
    args = {'test_id': 'P4_4_4_2_2',
        'version': version,
        'write_to_db': True,
        'last': last,
        'mode2': mode2,
        'gen_num': 5,
        'round_limit': 40,
        'db_file_name': "./db/metrics.db"
       }
    build_and_predict(args=args, lstm_args=lstm_args, sleep_time=3, verbose=False)
    time.sleep(10)
    usage = resource.getrusage(resource.RUSAGE_SELF)
    print(f"CPU 시간: {usage.ru_utime} 초")
    print(f"메모리 사용량: {usage.ru_maxrss} KB")
    clear_output_this_02 = False
    if clear_output_this_02:
        from IPython.display import clear_output
        clear_output() # Clears the output


********************
working model_version = 1  2025-05-28 10:51:21.502612
********************
start to train all models. 2025-05-28 10:51:21.502781
6's training. status=start 2025-05-28 10:51:21.502802
6's training. status=end 2025-05-28 10:51:30.250996
completed to train all models. 2025-05-28 10:51:30.251094
train to create model(1). 2025-05-28 10:51:33.251179
CPU 시간: 18.659042 초
메모리 사용량: 691792 KB
********************
working model_version = 2  2025-05-28 10:51:48.575884
********************
start to train all models. 2025-05-28 10:51:48.575920
6's training. status=start 2025-05-28 10:51:48.575936
6's training. status=end 2025-05-28 10:51:57.610481
completed to train all models. 2025-05-28 10:51:57.610579
train to create model(2). 2025-05-28 10:52:00.610665
CPU 시간: 33.405411 초
메모리 사용량: 712160 KB
********************
working model_version = 3  2025-05-28 10:52:15.912274
********************
start to train all models. 2025-05-28 10:52:15.912311
6's training. status=start 2025-05-28 

In [12]:
view_model_info = False
if view_model_info:
    for i in range(MAX_MODEL_CNT):
        print(models[i][0].model.summary())

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()